In [24]:
%tensorflow_version 2.x 

In [25]:
!pip install -q sklearn

In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import pandas as pd

from IPython.display import clear_output


In [27]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [28]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [29]:
train_y = train.pop('Species')
test_y = test.pop('Species')

train.shape

(120, 4)

In [30]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [31]:
train.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [32]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [33]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Three hidden layers of 20 and 30 and 10 nodes respectively.
    hidden_units=(20,30, 10),
    # The model must choose between 3 classes.
    n_classes=3)
clear_output()

In [34]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpf7jmthq4/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2939639, step = 0
INFO:tensorflow:global_step/sec: 484.071
INFO:tensorflow:loss = 1.0682521, step = 100 (0.208 sec)
INFO:tensorflow:global_step/sec: 641.285
INFO:tensorflow:loss = 1.0238295, step = 200 (0.158 sec)
INFO:tensorflow:global_step/sec: 545.72
INFO:tensorflow:loss = 0.99512684, step = 300 (0.183 sec)
INFO:tensorflow:global_step/sec: 603.3
INFO:tensorflow:loss = 0.96089387, step = 400 (0.166 sec)
INFO:tensorflow:global_step/sec: 613.889
INFO:tensorflow:loss = 0.92482805, step = 500 (0.164 sec)
INFO:t

In [35]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-16T06:50:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpf7jmthq4/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20428s
INFO:tensorflow:Finished evaluation at 2021-05-16-06:50:03
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.52417874, global_step = 5000, loss = 0.52417874
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpf7jmthq4/model.ckpt-5000

Test set accuracy: 0.700



In [36]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [37]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [42]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))
for pred_dict in predictions:
  print(pred_dict)

predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpf7jmthq4/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 2.4480145 ,  0.46274915, -0.43001953], dtype=float32), 'probabilities': array([0.8378091 , 0.11506819, 0.04712276], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
{'logits': array([-0.44456133,  1.4719027 ,  1.4148048 ], dtype=float32), 'probabilities': array([0.07034056, 0.47809651, 0.4515629 ], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
{'logits': array([-1.759142 ,  1.7547351,  2.344736 ], dtype=float32), 'probabilities': arra

In [43]:
pd.DataFrame({
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
})

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.1,3.3,1.7,0.5
1,5.9,3.0,4.2,1.5
2,6.9,3.1,5.4,2.1
